In [8]:
import numpy as np
from itertools import product
import pandas as pd
import pickle
import os
import foraging_toolkit as ft
import environments
import simulation
import success_metrics
import imp
imp.reload(ft)
imp.reload(simulation)
imp.reload(environments)
imp.reload(success_metrics)
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

Parameters of the forward model. Specify the parameter values for all the cases you are interested in. 

In [9]:
#These parameter values correspond to what is used in the CVPR paper, to the best of my knowledge
c_trust = np.array([0, 0.5])
sight_radius = [5]
reward_patch_dim = [1, 4] # clustered is 4, distributed is 1

# List of all possible combination of parameter values 
param_list = [i for i in product(c_trust, sight_radius, reward_patch_dim)]

print(len(param_list))
print('param_list', param_list)




4
param_list [(0.0, 5, 1), (0.0, 5, 4), (0.5, 5, 1), (0.5, 5, 4)]


Generate a dataframe containing all possible combinations of the parameter values specified above.

In [10]:
#each row of both dataframes corresponds to one experiment.  
#columns are parameters 
metadataDF = pd.DataFrame(param_list)

# success_metricsDF = pd.DataFrame(results)
# allresultsDF = pd.concat([metadataDF, success_metricsDF], axis='columns') # concatenate metadata and results 
metadataDF.columns = ["c_trust", "sight_radius", "reward_patch_dim"] 
metadataDF['sim index'] = np.arange(len(metadataDF)).astype(int)
metadataDF.head()
display(metadataDF)

,c_trust,sight_radius,reward_patch_dim,sim index
0,0.0,5,1,0
1,0.0,5,4,1
2,0.5,5,1,2
3,0.5,5,4,3


In [11]:
# select a specific row of the data frame
ind = 1
row = metadataDF.iloc[[ind]]
print(row)
print(row['c_trust'].iloc[0])
print(row['sight_radius'].iloc[0])
print(row['reward_patch_dim'].iloc[0])

   c_trust  sight_radius  reward_patch_dim  sim index
1      0.0             5                 4          1
0.0
5
4


*Part 2*
Test run one of the simulations from metadata (one row)

In [16]:
# choose a location to save the results data
home_dir = "/Users/emily/code/collaborative-intelligence/tmp_files/" 

# parameters to add
N_runs = 2 # How many times would you like to run a simulation (row of metadataDF)
N_frames = 50
N_agents = 9
N_total_food_units = 16
edge_size = 30
N_sims = len(metadataDF)

# save these extra metatdata parameters into a dictionary 
dictionary = {'N_sims':N_sims, 'N_runs':N_runs, 'N_frames':N_frames, 'N_agents':N_agents, \
              'N_total_food_units':N_total_food_units, 'edge_size':edge_size}

pickle.dump(dictionary, open(home_dir+ 'additional_meta_params.sav', 'wb'))

# data frame to store all success values across all simulations and runs of each simulation
success_measures_allDF = pd.DataFrame() 
success_sims_list = []



# save metadata to home directory
metadataDF.to_csv(home_dir + '\metadataDF' + '.csv')

for si in range(N_sims):
    
    # 1. pull out parameters from row si in the metadata
    df_row = metadataDF.iloc[[si]]
    c_trust = df_row['c_trust'].iloc[0]
    sight_radius = df_row['sight_radius'].iloc[0]
    reward_patch_dim = df_row['reward_patch_dim'].iloc[0].astype(int)

    # print(df_row)
    # print(reward_patch_dim)

    # arrays to save success measures for each run of this simulation
    mean_times_to_first_reward = np.zeros((N_runs))
    num_birds_failed = np.zeros((N_runs))

    # Do multiple runs of the simulation and store the results in a results dataframe
    for ri in range(N_runs):
        #  Feed the parameters through the run() function once (later we'll do this many times)
        env = environments.Environment(edge_size=edge_size, N_total_food_units=N_total_food_units, patch_dim=reward_patch_dim)
        env.add_food_patches() #food_statistics_type="drop_food_once")
        sim = simulation.SimulateCommunicators(env, N_frames, N_agents, c_trust=c_trust, sight_radius=sight_radius)
        sim.run()
        # print(sim.all_birdsDF)
        # # print(sim.all_rewardsDF) 
        # # by this point we should have birdlocsDF and rewardlocsDF 
        

        # Compute success measures 
        
        time_to_first_allbirds = np.zeros(N_agents)
        for bird_id in range(1, N_agents + 1): # compute time to first food for each bird 
            singlebirdDF = sim.all_birdsDF.loc[sim.all_birdsDF['bird'] == bird_id]  
            time_to_first_allbirds[bird_id - 1] = success_metrics.compute_time_to_first_reward(singlebirdDF, sim.all_rewardsDF, N_frames)
            
        # these are the success measures
        mean_times_to_first_reward[ri] = np.mean(time_to_first_allbirds) # take the average across birds
        num_birds_failed[ri] = np.sum(time_to_first_allbirds == N_frames) # number of birds that failed to reach food
        # print(num_birds_failed)
        #

        # SAVE THE RAW DATA 
        # make a new folder  
        sim_folder = 'sim' + str(si) + '_run' + str(ri)
        sim_dir = home_dir + sim_folder
        if not os.path.isdir(sim_dir):
            os.makedirs(sim_dir)
            print("Directory %s was created." %sim_folder)

        birdfile = '\\birdlocsDF'
        rewardfile = '\\rewardlocsDF'
        sim.all_birdsDF.to_csv(sim_dir + birdfile + ".csv")
        sim.all_rewardsDF.to_csv(sim_dir + rewardfile + ".csv")

       

    #Combine the metadata and the success measures into the results dataframe 
        # create a new row in the results data frame, then concatenate 
    success_measures_onesim_DF = pd.DataFrame(
        {
            "run index": np.arange(N_runs),
            "time to first food": mean_times_to_first_reward,
            "num birds failed": num_birds_failed,
        }
    )

    success_sims_list.append(success_measures_onesim_DF)

success_measures_allDF = pd.concat(success_sims_list)

# print(len(success_sims_list))
# print(len(mean_times_to_first_reward))
# print(len(success_measures_onesim_DF))
print(display(success_measures_onesim_DF))
print(display(success_measures_allDF))

# Concatenate metadataDF and success_measures_allDF into a new dataframe called resultsDF

    
# Save the results dataframe into the same folder as the metadata


FileNotFoundError: [Errno 2] No such file or directory: 'tmp_files/additional_meta_params.sav'

Concatenate metadataDF and success_measures_allDF into a new dataframe called resultsDF

In [104]:
# First, duplicate each row (sim) of metadataDF by the number of runs
newDF = pd.DataFrame(np.repeat(metadataDF.values, N_runs, axis=0))
newDF.columns = metadataDF.columns
display(newDF)


,c_trust,sight_radius,reward_patch_dim,sim index
0,0.0,5.0,1.0,0.0
1,0.0,5.0,1.0,0.0
2,0.0,5.0,4.0,1.0
3,0.0,5.0,4.0,1.0
4,0.5,5.0,1.0,2.0
5,0.5,5.0,1.0,2.0
6,0.5,5.0,4.0,3.0
7,0.5,5.0,4.0,3.0


In [105]:
# change the indexing of the rows of this dataframe so we can concatenate with metadataDF
successDF = success_measures_allDF.reset_index(drop=True)
display(successDF)

,run index,time to first food,num birds failed
0,0,2.888889,0.0
1,1,7.000000,1.0
2,0,33.666667,5.0
3,1,44.888889,8.0
4,0,1.666667,0.0
5,1,7.333333,1.0
6,0,39.000000,6.0
7,1,29.444444,4.0


In [106]:
# Concatenate metadataDF and successDF
allresultsDF = pd.concat([newDF, successDF], axis=1)
display(allresultsDF)

,c_trust,sight_radius,reward_patch_dim,sim index,run index,time to first food,num birds failed
0,0.0,5.0,1.0,0.0,0,2.888889,0.0
1,0.0,5.0,1.0,0.0,1,7.000000,1.0
2,0.0,5.0,4.0,1.0,0,33.666667,5.0
3,0.0,5.0,4.0,1.0,1,44.888889,8.0
4,0.5,5.0,1.0,2.0,0,1.666667,0.0
5,0.5,5.0,1.0,2.0,1,7.333333,1.0
6,0.5,5.0,4.0,3.0,0,39.000000,6.0
7,0.5,5.0,4.0,3.0,1,29.444444,4.0


In [107]:
# save metadata to home directory
allresultsDF.to_csv(home_dir + 'resultsDF' + '.csv')

Test one of the simulations by animating it.

In [108]:
communicators = ft.object_from_data(sim.all_birdsDF, sim.all_rewardsDF)
ft.animate_birds(communicators, plot_rewards=True,
                  width = 600, height = 600, point_size = 10)

In [109]:
print(sim.all_birdsDF.loc[sim.all_birdsDF['bird'] == 1])

       x     y  time  bird           type
0   26.0  12.0     1     1  communicators
1   24.0  13.0     2     1  communicators
2   22.0  15.0     3     1  communicators
3   24.0  14.0     4     1  communicators
4   26.0  13.0     5     1  communicators
5   24.0  12.0     6     1  communicators
6   26.0  11.0     7     1  communicators
7   26.0   8.0     8     1  communicators
8   24.0   7.0     9     1  communicators
9   23.0   7.0    10     1  communicators
10  21.0   9.0    11     1  communicators
11  18.0   9.0    12     1  communicators
12  18.0   9.0    13     1  communicators
13  18.0   9.0    14     1  communicators
14  18.0   9.0    15     1  communicators
15  18.0   9.0    16     1  communicators
16  18.0   9.0    17     1  communicators
17  18.0   9.0    18     1  communicators
18  18.0   9.0    19     1  communicators
19  18.0   9.0    20     1  communicators
20  18.0   9.0    21     1  communicators
21  18.0   9.0    22     1  communicators
22  18.0   9.0    23     1  commun

In [110]:
# print(sim.all_rewardsDF[]) 
print(sim.all_rewardsDF.loc[sim.all_rewardsDF['time'] == 4])


     x   y  time
0   15   8     4
1   16   8     4
2   17   8     4
3   18   8     4
4   15   9     4
5   16   9     4
6   17   9     4
7   18   9     4
8   15  10     4
9   16  10     4
10  17  10     4
11  18  10     4
12  15  11     4
13  16  11     4
14  17  11     4
15  18  11     4
